In [ ]:
pip install requests beautifulsoup4 pandas lxml openpyxl python-dateutil tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ai_adoption_crawler.py
# 목적: 은행별 공식/준공식 보도자료에서 AI(인공지능) + (신용평가/대출심사/자동심사/FDS/이상거래) 키워드를 포함한 문서를 수집
#      -> 근거 URL/제목/날짜/스니펫을 저장하고, 은행별 최초 연도를 후보로 추출
#
# 출력:
#   outputs/raw_hits.csv
#   outputs/raw_hits.xlsx
#   outputs/adoption_year_candidates.csv

from __future__ import annotations

import os
import re
import time
import json
import hashlib
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
from urllib.parse import urljoin, urlparse, parse_qs

import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil import parser as dateparser
from tqdm import tqdm

# ----------------------------
# 0) 공통 설정
# ----------------------------

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
}

AI_KEYWORDS = ["AI", "인공지능", "Artificial Intelligence"]
CREDIT_KEYWORDS = ["신용평가", "대출심사", "자동심사", "CSS", "FDS", "이상거래", "AML", "리스크", "credit scoring", "underwriting"]

# 키워드 동시충족 조건:
#  - AI키워드 >=1 AND (대출/신용/FDS 계열 키워드)>=1
def keyword_hit(text: str) -> Tuple[bool, List[str], List[str]]:
    t = text or ""
    ai_hits = [k for k in AI_KEYWORDS if k.lower() in t.lower()]
    cr_hits = [k for k in CREDIT_KEYWORDS if k.lower() in t.lower()]
    return (len(ai_hits) > 0 and len(cr_hits) > 0), ai_hits, cr_hits

def normalize_space(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()

def safe_filename(url: str) -> str:
    h = hashlib.md5(url.encode("utf-8")).hexdigest()[:12]
    return h

def parse_date_any(s: str) -> Optional[str]:
    """다양한 날짜표현에서 YYYY-MM-DD로 정규화"""
    if not s:
        return None
    s = s.strip()
    # 흔한 형식: 2026.02.13 / 2026-02-13 / 2026/02/13
    m = re.search(r"(20\d{2})[.\-/](\d{1,2})[.\-/](\d{1,2})", s)
    if m:
        y, mo, d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        return f"{y:04d}-{mo:02d}-{d:02d}"
    # 파싱 시도
    try:
        dt = dateparser.parse(s, fuzzy=True)
        if dt:
            return dt.date().isoformat()
    except Exception:
        pass
    return None

def extract_year(date_iso: Optional[str]) -> Optional[int]:
    if not date_iso:
        return None
    try:
        return int(date_iso[:4])
    except Exception:
        return None

def log(msg: str):
    path = os.path.join(OUTPUT_DIR, "run_log.txt")
    with open(path, "a", encoding="utf-8") as f:
        f.write(msg + "\n")

# ----------------------------
# 1) 크롤러 베이스
# ----------------------------

@dataclass
class Article:
    bank: str
    url: str
    title: str
    date: Optional[str]   # ISO
    text: str
    ai_hits: List[str]
    credit_hits: List[str]
    snippet: str

class BaseCrawler:
    bank: str

    def __init__(self, bank: str, session: Optional[requests.Session] = None):
        self.bank = bank
        self.sess = session or requests.Session()
        self.sess.headers.update(HEADERS)

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        """
        returns list of tuples: (url, title_hint, date_hint)
        """
        raise NotImplementedError

    def fetch_article(self, url: str, title_hint: Optional[str] = None, date_hint: Optional[str] = None) -> Article:
        """
        기본: HTML -> text 추출.
        사이트별로 override 가능.
        """
        r = self.sess.get(url, timeout=30)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "lxml")

        # title 추정
        title = title_hint or ""
        if not title:
            og = soup.select_one('meta[property="og:title"]')
            if og and og.get("content"):
                title = og.get("content").strip()
        if not title:
            if soup.title:
                title = soup.title.get_text(strip=True)

        # date 추정
        date_iso = parse_date_any(date_hint) if date_hint else None
        if not date_iso:
            # 흔한 패턴들
            # - meta property="article:published_time"
            meta_dt = soup.select_one('meta[property="article:published_time"]')
            if meta_dt and meta_dt.get("content"):
                date_iso = parse_date_any(meta_dt.get("content"))
        if not date_iso:
            # 페이지 text에서 날짜 패턴 탐색
            text_all = soup.get_text(" ", strip=True)
            m = re.search(r"(20\d{2})[.\-/](\d{1,2})[.\-/](\d{1,2})", text_all)
            if m:
                date_iso = f"{int(m.group(1)):04d}-{int(m.group(2)):02d}-{int(m.group(3)):02d}"

        # 본문 text
        text = soup.get_text("\n", strip=True)
        text = normalize_space(text)

        hit, ai_hits, cr_hits = keyword_hit(text)

        # 스니펫: 첫 매칭 위치 주변 250자
        snippet = ""
        if hit:
            # 가장 먼저 등장하는 키워드 위치 찾기
            idxs = []
            for k in (ai_hits + cr_hits):
                i = text.lower().find(k.lower())
                if i >= 0:
                    idxs.append(i)
            if idxs:
                i0 = max(min(idxs) - 120, 0)
                i1 = min(min(idxs) + 250, len(text))
                snippet = text[i0:i1]

        return Article(
            bank=self.bank,
            url=url,
            title=title or "",
            date=date_iso,
            text=text,
            ai_hits=ai_hits,
            credit_hits=cr_hits,
            snippet=snippet
        )

# ----------------------------
# 2) 은행별 크롤러들
# ----------------------------

class KBCrawler(BaseCrawler):
    """
    KB국민은행 보도자료 (omoney.kbstar.com/quics?page=C017648) 기반
    """
    BASE = "https://omoney.kbstar.com/quics?page=C017648"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        # viewPage=1,2,... 페이지가 있을 수 있어 범위를 넉넉히 탐색
        for view_page in range(1, 60):  # 필요시 증가
            list_url = f"{self.BASE}&bbsMode=list&boardId=647&viewPage={view_page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")
            rows = soup.select("table tr")
            if not rows:
                break

            found_any = False
            for tr in rows:
                a = tr.find("a")
                tds = tr.find_all("td")
                if a and a.get("href") and len(tds) >= 3:
                    href = a.get("href")
                    title = a.get_text(strip=True)
                    date_hint = tds[-2].get_text(strip=True)  # 등록일 위치가 보통 뒤쪽
                    full = urljoin("https://omoney.kbstar.com", href)
                    urls.append((full, title, date_hint))
                    found_any = True
            if not found_any:
                break
            time.sleep(0.2)
        return urls

class ShinhanGroupCrawler(BaseCrawler):
    """
    신한금융그룹 그룹사 뉴스(신한은행 포함)
    https://www.shinhangroup.com/kr/archive/press
    """
    BASE = "https://www.shinhangroup.com/kr/archive/press"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        # 페이지 구조가 변할 수 있어: 일단 여러 페이지를 내려가며 a태그 추출
        for page in range(1, 80):
            list_url = f"{self.BASE}?page={page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            # 카드형 목록에서 링크 추출
            items = soup.select("a")
            got = 0
            for a in items:
                href = a.get("href") or ""
                txt = a.get_text(" ", strip=True)
                if "[신한은행]" in txt and href:
                    full = urljoin(self.BASE, href)
                    # 날짜 힌트: 주변 텍스트에서 찾기
                    date_hint = None
                    # 같은 카드 내 날짜를 찾기 시도
                    parent = a.parent
                    if parent:
                        ptxt = parent.get_text(" ", strip=True)
                        date_hint = parse_date_any(ptxt)
                    urls.append((full, txt, date_hint))
                    got += 1
            if got == 0:
                # 더 이상 없으면 중단
                break
            time.sleep(0.2)
        return urls

class WooriCrawler(BaseCrawler):
    """
    우리은행 보도자료 (spot.wooribank.com)
    https://spot.wooribank.com/pot/Dream?withyou=BPPBC0036
    """
    BASE = "https://spot.wooribank.com/pot/Dream?withyou=BPPBC0036"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        for page in range(1, 80):
            list_url = f"{self.BASE}&page={page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            # 보도자료 목록에서 a 태그
            anchors = soup.select("a")
            got = 0
            for a in anchors:
                href = a.get("href") or ""
                title = a.get_text(" ", strip=True)
                if "Dream?withyou=" in href and "BPPBC" in href and title:
                    full = urljoin("https://spot.wooribank.com", href)
                    # 날짜 힌트: title 옆 괄호( 2026.01.15 ) 같은 게 있을 때
                    date_hint = None
                    m = re.search(r"(20\d{2}[.\-/]\d{1,2}[.\-/]\d{1,2})", title)
                    if m:
                        date_hint = m.group(1)
                    urls.append((full, title, date_hint))
                    got += 1
            if got == 0:
                break
            time.sleep(0.2)
        return urls

class NHBankCrawler(BaseCrawler):
    """
    NH농협은행 NH뉴스
    https://www.nhbank.com/goSubPage.do?srchGb=KO_NHPR_02&srchSiteGb=KR
    """
    BASE = "https://www.nhbank.com/goSubPage.do?srchGb=KO_NHPR_02&srchSiteGb=KR"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        for page in range(1, 120):
            list_url = f"{self.BASE}&srchP={page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            anchors = soup.select("a")
            got = 0
            for a in anchors:
                href = a.get("href") or ""
                title = a.get_text(" ", strip=True)
                if ("goSubPage.do" in href or "goSubPage" in href) and title:
                    full = urljoin("https://www.nhbank.com", href)
                    # 날짜는 카드 텍스트에서 찾기
                    parent = a.parent
                    date_hint = None
                    if parent:
                        date_hint = parse_date_any(parent.get_text(" ", strip=True))
                    urls.append((full, title, date_hint))
                    got += 1
            if got == 0:
                break
            time.sleep(0.2)
        return urls

class HanaFNCrawler(BaseCrawler):
    """
    하나금융 PR센터(그룹)에서 #하나은행 태그 뉴스만 추출
    https://www.hanafn.com/mediaRoom/mediaRoom.do
    """
    BASE = "https://www.hanafn.com/mediaRoom/mediaRoom.do"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        for page in range(1, 100):
            list_url = f"{self.BASE}?pageIndex={page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            got = 0
            # 카드 텍스트에 "#하나은행" 포함된 항목 찾기
            for a in soup.select("a"):
                href = a.get("href") or ""
                txt = a.get_text(" ", strip=True)
                if "하나은행" in txt and href:
                    full = urljoin("https://www.hanafn.com", href)
                    # 날짜 힌트: 근처에 2026.02.02 등
                    date_hint = None
                    parent = a.parent
                    if parent:
                        date_hint = parse_date_any(parent.get_text(" ", strip=True))
                    urls.append((full, txt, date_hint))
                    got += 1

            if got == 0:
                break
            time.sleep(0.2)
        return urls

class KakaoBankCrawler(BaseCrawler):
    """
    카카오뱅크 보도자료
    https://www.kakaobank.com/Corp/News/PressRelease/pages/1
    """
    BASE = "https://www.kakaobank.com/Corp/News/PressRelease/pages/"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        for page in range(1, 200):
            list_url = f"{self.BASE}{page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            got = 0
            for a in soup.select("a"):
                href = a.get("href") or ""
                title = a.get_text(" ", strip=True)
                if "/Corp/News/PressRelease/" in href and href.endswith(tuple("0123456789")):
                    full = urljoin("https://www.kakaobank.com", href)
                    # 날짜 힌트: 카드 내에 2023.04.11 같은 표기가 있음
                    date_hint = None
                    parent = a.parent
                    if parent:
                        date_hint = parse_date_any(parent.get_text(" ", strip=True))
                    urls.append((full, title, date_hint))
                    got += 1
            if got == 0:
                break
            time.sleep(0.2)
        return urls

class TossbankWriterCrawler(BaseCrawler):
    """
    토스피드 writer/tossbank (토스뱅크 공식 writer 페이지)
    https://toss.im/tossfeed/writer/tossbank
    """
    BASE = "https://toss.im/tossfeed/writer/tossbank"

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        # 페이지네이션이 SPA 형태일 수 있어, 우선 단순 페이지 파라미터 시도
        # 안 될 경우(기사 일부만 수집) selenium 확장 필요
        for page in range(1, 40):
            list_url = f"{self.BASE}?page={page}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            got = 0
            for a in soup.select("a"):
                href = a.get("href") or ""
                title = a.get_text(" ", strip=True)
                if "/tossfeed/article/" in href and title:
                    full = urljoin("https://toss.im", href)
                    # 날짜 힌트
                    date_hint = None
                    parent = a.parent
                    if parent:
                        date_hint = parse_date_any(parent.get_text(" ", strip=True))
                    urls.append((full, title, date_hint))
                    got += 1
            if got == 0:
                # 더 없으면 종료
                break
            time.sleep(0.2)
        return urls

class FSCPressCrawler(BaseCrawler):
    """
    금융위원회 보도자료 검색 기반 (케이뱅크 등)
    예: https://www.fsc.go.kr/no010101/81348?... (개별 문서)
    검색 목록은 no010101?srchText=...
    """
    SEARCH_BASE = "https://www.fsc.go.kr/no010101"

    def __init__(self, bank: str, query: str, session: Optional[requests.Session] = None):
        super().__init__(bank, session=session)
        self.query = query

    def list_article_urls(self) -> List[Tuple[str, Optional[str], Optional[str]]]:
        urls = []
        for page in range(1, 80):
            list_url = f"{self.SEARCH_BASE}?curPage={page}&srchKey=all&srchText={requests.utils.quote(self.query)}"
            r = self.sess.get(list_url, timeout=30)
            if r.status_code != 200:
                break
            soup = BeautifulSoup(r.text, "lxml")

            got = 0
            for a in soup.select("a"):
                href = a.get("href") or ""
                title = a.get_text(" ", strip=True)
                # 보도자료 상세는 보통 /no010101/숫자 형태
                if re.search(r"/no010101/\d+", href) and title:
                    full = urljoin("https://www.fsc.go.kr", href)
                    # 날짜 힌트: 목록 주변 텍스트에서 찾기
                    date_hint = None
                    parent = a.parent
                    if parent:
                        date_hint = parse_date_any(parent.get_text(" ", strip=True))
                    urls.append((full, title, date_hint))
                    got += 1
            if got == 0:
                break
            time.sleep(0.2)
        return urls

# ----------------------------
# 3) 실행 함수
# ----------------------------

def run_all(limit_each: Optional[int] = None) -> pd.DataFrame:
    session = requests.Session()
    session.headers.update(HEADERS)

    crawlers: List[BaseCrawler] = [
        KBCrawler("국민은행", session=session),
        ShinhanGroupCrawler("신한은행", session=session),
        WooriCrawler("우리은행", session=session),
        NHBankCrawler("농협은행", session=session),
        HanaFNCrawler("하나은행", session=session),
        KakaoBankCrawler("카카오뱅크", session=session),
        TossbankWriterCrawler("토스뱅크", session=session),

        # 케이뱅크: 은행 자체 보도자료 아카이브가 공개형으로 명확히 확인되지 않아
        # "정부(금융위원회) 보도자료"에서 '케이뱅크'를 검색하여 공식 근거 확보
        FSCPressCrawler("케이뱅크", query="케이뱅크", session=session),

        # 기업은행: 공식 홈페이지 PR 아카이브가 크롤링 난이도/차단 이슈가 있어,
        # 연구 재현성을 위해 우선 보도자료 배포 플랫폼(뉴스와이어)도 함께 쓰는 방식 권장.
        # (원하면 IBK 전용 selenium 버전 추가 제공 가능)
        # 여기서는 예시로 금융위 검색도 함께 추가(공식 문서 기준)
        FSCPressCrawler("기업은행", query="기업은행", session=session),
    ]

    hits: List[Article] = []
    log(f"[RUN] start at {time.strftime('%Y-%m-%d %H:%M:%S')}")

    for crawler in crawlers:
        log(f"[CRAWLER] {crawler.bank} listing...")
        try:
            items = crawler.list_article_urls()
        except Exception as e:
            log(f"[ERROR] list failed: {crawler.bank} :: {e}")
            continue

        if limit_each:
            items = items[:limit_each]

        log(f"[CRAWLER] {crawler.bank} listed {len(items)} urls")

        for (url, title_hint, date_hint) in tqdm(items, desc=f"Fetch {crawler.bank}", leave=False):
            try:
                art = crawler.fetch_article(url, title_hint=title_hint, date_hint=date_hint)
                ok, _, _ = keyword_hit(art.text)
                if ok:
                    hits.append(art)

                    # 원문 증거를 남기기 위해 html/text 일부도 저장(텍스트만)
                    fn = safe_filename(url)
                    with open(os.path.join(OUTPUT_DIR, f"{crawler.bank}_{fn}.txt"), "w", encoding="utf-8") as f:
                        f.write(art.text)

            except Exception as e:
                log(f"[ERROR] fetch failed: {crawler.bank} :: {url} :: {e}")
                continue

    log(f"[RUN] done at {time.strftime('%Y-%m-%d %H:%M:%S')}, hits={len(hits)}")

    # dataframe 변환
    df = pd.DataFrame([{
        "bank": a.bank,
        "date": a.date,
        "year": extract_year(a.date),
        "title": a.title,
        "url": a.url,
        "ai_hits": ", ".join(a.ai_hits),
        "credit_hits": ", ".join(a.credit_hits),
        "snippet": a.snippet
    } for a in hits]).sort_values(["bank", "date", "title"], na_position="last")

    df.to_csv(os.path.join(OUTPUT_DIR, "raw_hits.csv"), index=False, encoding="utf-8-sig")
    df.to_excel(os.path.join(OUTPUT_DIR, "raw_hits.xlsx"), index=False)

    # 은행별 후보(최초 연도)
    cand = (
        df.dropna(subset=["year"])
          .groupby("bank")["year"]
          .min()
          .reset_index()
          .rename(columns={"year": "AI_adopt_year_candidate"})
          .sort_values("bank")
    )
    cand.to_csv(os.path.join(OUTPUT_DIR, "adoption_year_candidates.csv"), index=False, encoding="utf-8-sig")

    return df

if __name__ == "__main__":
    # limit_each를 None으로 두면 가능한 범위를 최대 탐색
    # 처음 디버깅은 limit_each=50 같은 식으로 추천
    run_all(limit_each=None)
    print("Done. Check outputs/ folder.")

Done. Check outputs/ folder.


In [ ]:
import re
import pandas as pd
import numpy as np

# 1) 로드
df = pd.read_csv("raw_hits.csv")

# 2) 공백/NA 정리
for c in ["bank","date","year","title","url","ai_hits","credit_hits","snippet"]:
    if c in df.columns:
        df[c] = df[c].astype(str).replace({"nan": "", "None": ""}).str.strip()

# 3) URL 없는 행 제거 (핵심키)
df = df[df["url"].ne("")].copy()

# 4) date 파싱 + year 재생성(우선 date가 멀쩡한 것만)
df["date_parsed"] = pd.to_datetime(df["date"], errors="coerce")
df["year_from_date"] = df["date_parsed"].dt.year

# 기존 year가 비었거나 이상하면 date 기반으로 교체
df["year_num"] = pd.to_numeric(df["year"], errors="coerce")
bad_year = df["year_num"].isna() | (df["year_num"] < 1990) | (df["year_num"] > 2030)
df.loc[bad_year & df["year_from_date"].notna(), "year_num"] = df.loc[bad_year & df["year_from_date"].notna(), "year_from_date"]

# 5) hit 컬럼 정규화: "AI, 인공지능" 같은 텍스트를 키워드 플래그로
def norm_text(x):
    x = (x or "").strip()
    x = re.sub(r"\s+", " ", x)
    return x

df["ai_hits_norm"] = df["ai_hits"].map(norm_text)
df["credit_hits_norm"] = df["credit_hits"].map(norm_text)

AI_KW = ["AI", "인공지능", "머신러닝", "딥러닝", "생성형", "Gen AI", "챗봇"]
CREDIT_KW = ["신용평가", "대출심사", "리스크", "AML", "FDS", "이상거래", "자금세탁"]

def has_kw(text, kws):
    t = (text or "")
    return any(k.lower() in t.lower() for k in kws)

df["has_ai"] = df["ai_hits_norm"].apply(lambda t: has_kw(t, AI_KW))
df["has_credit_related"] = df["credit_hits_norm"].apply(lambda t: has_kw(t, CREDIT_KW))

# 6) snippet 정리: 공통 메뉴/푸터 제거(농협 경영공시 같은 케이스)
BOILER = [
    "찾아오시는 길", "영업점 안내", "윤리경영", "금융소비자보호", "민원", "FAQ", "새창열림",
    "CI 다운로드", "채용정보", "인사제도", "공시정보"
]
def clean_snippet(s):
    s = (s or "").strip()
    s = re.sub(r"\s+", " ", s)
    # 보일러플레이트가 과다 포함되면 빈값 처리
    hit = sum(1 for b in BOILER if b in s)
    if hit >= 3:
        return ""
    # 너무 길면 잘라두기
    return s[:400]

df["snippet_clean"] = df["snippet"].apply(clean_snippet)

# 7) 중복 제거: url 기준 (가장 강력)
df = df.sort_values(["url","date_parsed"], ascending=[True, False])
df = df.drop_duplicates(subset=["url"], keep="first").copy()

# 8) 농협 경영공시 같이 title이 너무 일반적인 것 필터(선택)
generic_title = df["title"].isin(["경영공시"])
df.loc[generic_title & df["snippet_clean"].eq(""), "drop_candidate"] = True
df["drop_candidate"] = df["drop_candidate"].fillna(False)

# 저장
df.to_csv("clean_step1.csv", index=False, encoding="utf-8-sig")
print(df.shape)


In [ ]:
pip install pdfplumber pandas tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import re
import glob
import pandas as pd

try:
    import pdfplumber
except ImportError:
    raise SystemExit("pdfplumber가 없습니다. conda/pip로 설치하세요: pip install pdfplumber")

# =========================
# 1) 너 환경에 맞게 수정
# =========================
ROOT_DIR = r"C:\Users\DS\Downloads"   # <- PDF들 있는 폴더(하위폴더 포함 스캔)
OUTPUT_CSV = os.path.join(ROOT_DIR, "ai_credit_extraction_results.csv")

# 은행 9개 (너가 말한 기준)
BANKS = [
    "국민은행", "신한은행", "우리은행", "기업은행", "하나은행",
    "케이뱅크", "카카오뱅크", "토스뱅크", "농협은행"
]

COM = [
    "농협금융", "하나금융", "우리금융"
]

YEARS = [2019, 2020, 2021, 2022, 2023, 2024]

# 파일명 패턴(너가 말한 룰 반영)
# - 농협/토스: 은행명_감사보고서2019 (기간 2019~2024)
# - 그 외: 은행명_사업보고서_연도
# - 지속가능: 은행명_지속가능경영_연도
PATTERNS = [
    "{bank}_사업보고서{year}*.pdf",
    "{bank}_지속가능경영{year}*.pdf",
    "{com}_지속가능경영{year}*.pdf",
    "{bank}_감사보고서{year}*.pdf",
    "{bank}_감사보고서{year}*.PDF",
    "{bank}_사업보고서{year}*.PDF",
    "{bank}_지속가능경영{year}*.PDF",
    "{com}_지속가능경영{year}*.PDF",
]

# =========================
# 2) 탐지 규칙 (A 정의)
# =========================
# 'AI' 오탐 방지: 단어 경계로 AI만
AI_TOKEN = re.compile(r"\bAI\b", re.IGNORECASE)

# 대출심사/신용평가/리스크관리 키워드(한국어+영문 일부)
DOMAIN_KEYWORDS = [
    r"대출", r"여신", r"심사", r"신용", r"평가", r"스코어", r"스코어링",
    r"리스크", r"위험", r"신용위험", r"시장위험", r"운영위험",
    r"FDS", r"AML", r"이상거래", r"부정거래", r"사기", r"Fraud",
    r"신용평가모형", r"리스크관리", r"Risk"
]
DOMAIN_RE = re.compile("|".join(DOMAIN_KEYWORDS), re.IGNORECASE)

# 운영/상용화 여부를 강하게 시사하는 단어(있으면 더 "확실")
OPERATION_HINTS = [
    r"운영", r"상용", r"적용\s*중", r"도입\s*후", r"전사", r"확대\s*적용",
    r"고도화", r"활용\s*중", r"적용", r"구축\s*완료"
]
OP_RE = re.compile("|".join(OPERATION_HINTS), re.IGNORECASE)

# PoC/시범/개발중이면 0으로 떨어뜨릴 때 사용(근거에 표시)
PILOT_HINTS = [
    r"시범", r"파일럿", r"PoC", r"검증", r"테스트", r"개발\s*중", r"추진", r"예정"
]
PILOT_RE = re.compile("|".join(PILOT_HINTS), re.IGNORECASE)

# 문장 분리
SENT_SPLIT = re.compile(r"(?:[\.?!。]+|다\.)\s*|[\n\r]+")

# AI와 도메인 키워드가 가까이 있는지 확인할 윈도우(문장 단위로 이미 좁지만 추가 안전장치)
MAX_CHAR_WINDOW = 250  # 같은 문장 내에서 AI 주변 250자 내 도메인키워드 있으면 인정

# =========================
# 3) PDF 텍스트 추출 함수
# =========================
def extract_text_from_pdf(pdf_path: str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            t = page.extract_text() or ""
            if t.strip():
                texts.append(t)
    return "\n".join(texts)

def find_evidence_sentences(text: str):
    """
    반환: (evidence_list)
    evidence_list item:
      dict(sentence, has_op, has_pilot, ai_pos, domain_pos)
    """
    evidences = []
    sentences = [s.strip() for s in SENT_SPLIT.split(text) if s and s.strip()]
    for s in sentences:
        # AI 토큰 먼저 체크
        m_ai = AI_TOKEN.search(s)
        if not m_ai:
            continue

        # 같은 문장에 도메인 키워드가 있는지(혹은 AI 주변 윈도우 내)
        # 1) 문장 전체에 도메인 키워드
        if DOMAIN_RE.search(s):
            ok = True
        else:
            ok = False

        # 2) 추가로 AI 주변 window 검사(문장 내에서만)
        if not ok:
            ai_i = m_ai.start()
            lo = max(0, ai_i - MAX_CHAR_WINDOW)
            hi = min(len(s), ai_i + MAX_CHAR_WINDOW)
            if DOMAIN_RE.search(s[lo:hi]):
                ok = True

        if not ok:
            continue

        evidences.append({
            "sentence": s,
            "has_operation_hint": bool(OP_RE.search(s)),
            "has_pilot_hint": bool(PILOT_RE.search(s)),
        })
    return evidences

# =========================
# 4) 파일 수집
# =========================
def collect_files():
    files = []
    for bank in BANKS:
        for year in YEARS:
            for com in COM:
                for pat in PATTERNS:
                    g = os.path.join(ROOT_DIR, "**", pat.format(bank=bank, year=year, com=com))
                    files.extend(glob.glob(g, recursive=True))
    # 중복 제거
    files = sorted(list(dict.fromkeys(files)))
    return files

def infer_bank_year_from_filename(path: str):
    base = os.path.basename(path)
    bank = next((b for b in BANKS if b in base), None)
    year = None
    for y in YEARS:
        if str(y) in base:
            year = y
            break
    doc_type = None
    if "사업보고서" in base:
        doc_type = "사업보고서"
    elif "감사보고서" in base:
        doc_type = "감사보고서"
    elif "지속가능경영" in base:
        doc_type = "지속가능경영"
    else:
        doc_type = "기타"
    return bank, year, doc_type

# =========================
# 5) 메인 실행
# =========================
def main():
    pdf_files = collect_files()
    if not pdf_files:
        print("❌ 매칭되는 PDF 파일을 못 찾았어. ROOT_DIR/파일명 패턴 확인해줘.")
        print("현재 ROOT_DIR:", ROOT_DIR)
        return

    rows = []
    for i, pdf_path in enumerate(pdf_files, 1):
        bank, year, doc_type = infer_bank_year_from_filename(pdf_path)
        if bank is None or year is None:
            # 파일명이 규칙과 다르면 일단 스킵하지 말고 기록은 남김
            bank = bank or "미확인"
            year = year or "미확인"

        print(f"[{i}/{len(pdf_files)}] 읽는 중: {os.path.basename(pdf_path)}")

        try:
            text = extract_text_from_pdf(pdf_path)
        except Exception as e:
            rows.append({
                "bank": bank, "year": year, "doc_type": doc_type,
                "pdf_path": pdf_path, "ai_credit_flag": 0,
                "confidence": "error",
                "evidence": "",
                "note": f"PDF 읽기 실패: {e}"
            })
            continue

        evidences = find_evidence_sentences(text)

        if not evidences:
            rows.append({
                "bank": bank, "year": year, "doc_type": doc_type,
                "pdf_path": pdf_path, "ai_credit_flag": 0,
                "confidence": "none",
                "evidence": "",
                "note": ""
            })
            continue

        # 판정 로직:
        # - 근거 문장 중 operation_hint가 있으면 high
        # - pilot_hint만 있고 operation_hint 없으면 low(일단 0으로 두고 note에 표시)
        has_op_any = any(ev["has_operation_hint"] for ev in evidences)
        has_pilot_any = any(ev["has_pilot_hint"] for ev in evidences)

        if has_op_any:
            flag = 1
            conf = "high"
            note = ""
        else:
            # 운영근거 없고 파일럿/개발중만 보이면 0 유지(너 규칙)
            flag = 0
            conf = "low"
            note = "AI+도메인 언급은 있으나 운영/상용 근거가 약함(파일럿/추진 가능)"

        # 증거는 최대 3개만 저장(너무 길어지는 것 방지)
        evidence_text = " | ".join([ev["sentence"] for ev in evidences[:3]])

        # 파일럿 힌트가 같이 있으면 note 강화
        if has_pilot_any and not has_op_any:
            note = (note + "; " if note else "") + "문장에 시범/파일럿/개발중 힌트 포함"

        rows.append({
            "bank": bank, "year": year, "doc_type": doc_type,
            "pdf_path": pdf_path,
            "ai_credit_flag": flag,
            "confidence": conf,
            "evidence": evidence_text,
            "note": note
        })

    df = pd.DataFrame(rows)
    df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
    print("\n✅ 완료! 결과 저장:", OUTPUT_CSV)

    # 은행별/연도별 요약도 같이 보여주기
    try:
        pivot = df.pivot_table(index=["bank"], columns=["year"], values="ai_credit_flag", aggfunc="max", fill_value=0)
        print("\n=== 은행-연도 요약(최대값) ===")
        print(pivot)
    except Exception:
        pass

if __name__ == "__main__":
    main()


[1/84] 읽는 중: 국민은행_사업보고서2019.pdf
[2/84] 읽는 중: 국민은행_사업보고서2020.pdf
[3/84] 읽는 중: 국민은행_사업보고서2021.pdf
[4/84] 읽는 중: 국민은행_사업보고서2022.pdf
[5/84] 읽는 중: 국민은행_사업보고서2023.pdf
[6/84] 읽는 중: 국민은행_사업보고서2024.pdf
[7/84] 읽는 중: 국민은행_지속가능경영2019.pdf
[8/84] 읽는 중: 국민은행_지속가능경영2020.pdf
[9/84] 읽는 중: 국민은행_지속가능경영2021.pdf
[10/84] 읽는 중: 국민은행_지속가능경영2022.pdf
[11/84] 읽는 중: 국민은행_지속가능경영2023.pdf
[12/84] 읽는 중: 국민은행_지속가능경영2024.pdf
[13/84] 읽는 중: 기업은행_사업보고서2019.pdf
[14/84] 읽는 중: 기업은행_사업보고서2020.pdf
[15/84] 읽는 중: 기업은행_사업보고서2021.pdf
[16/84] 읽는 중: 기업은행_사업보고서2022.pdf
[17/84] 읽는 중: 기업은행_사업보고서2023.pdf
[18/84] 읽는 중: 기업은행_사업보고서2024.pdf
[19/84] 읽는 중: 기업은행_지속가능경영2020.pdf
[20/84] 읽는 중: 기업은행_지속가능경영2021.pdf
[21/84] 읽는 중: 기업은행_지속가능경영2022.pdf
[22/84] 읽는 중: 기업은행_지속가능경영2023.pdf
[23/84] 읽는 중: 기업은행_지속가능경영2024.pdf
[24/84] 읽는 중: 농협금융_지속가능경영2021.pdf
[25/84] 읽는 중: 농협금융_지속가능경영2022.pdf
[26/84] 읽는 중: 농협은행_감사보고서2019.pdf
[27/84] 읽는 중: 농협은행_감사보고서2020.pdf
[28/84] 읽는 중: 농협은행_감사보고서2021.pdf
[29/84] 읽는 중: 농협은행_감사보고서2022.pdf
[30/84] 읽는 중: 농협은행_감사보고서2023.pdf
[31/84

In [ ]:
pip install requests beautifulsoup4 lxml pandas trafilatura feedparser tqdm dateparser rapidfuzz

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 54.6 MB/s  0:00:00
   ---------------------------------------- 0.0/837.9 kB ? eta -:--:--
   ---------------------------------------- 837.9/837.9 kB 29.9 MB/s  0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6104 sha256=82a2384497d609064e0153968f0f2add865810516c4c2e80f959bd85e71148fd
  Stored in directory: c:\users\ds\appdata\local\pip\cache\wheels\3d\4d\ef\37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k

   ------- --------------------------------  2/11 [tld]
   --------

In [ ]:
import re
import time
import json
import urllib.parse
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import feedparser
import dateparser
from rapidfuzz import fuzz
import trafilatura


# =========================
# 1) 확정한 AI_credit 기준
# =========================

INCLUDE_KEYWORDS = [
    "인공지능 기반 신용평가",
    "AI 기반 대출심사",
    "자동심사 시스템",
    "머신러닝 기반 리스크관리",
    "FDS 고도화",
    "AML AI 시스템",
]

AI_TOKENS = ["AI", "인공지능", "머신러닝", "딥러닝", "ML", "학습모델"]
CREDIT_RISK_TOKENS = [
    "신용평가", "신용", "대출", "여신", "심사", "스코어링", "리스크", "위험", "리스크관리",
    "이상거래", "FDS", "자금세탁", "AML", "제재", "컴플라이언스", "사기탐지"
]

EXCLUDE_KEYWORDS = [
    "AI 상담",
    "AI 챗봇",
    "챗봇",
    "고객 응대",
    "디지털 창구",
    "모바일 상담",
    "콜센터",
    "FAQ",
]

# 도입연도 정의 = “상용화/운영 중이라고 명시된 최초 연도”
OPS_TRIGGERS = [
    "상용화", "운영", "운영 중", "적용 중", "도입 후 운영", "가동", "활용 중", "실서비스", "본격 운영"
]
NON_OPS_TRIGGERS = [
    "PoC", "시범", "파일럿", "검증", "개발", "구축", "예정", "추진", "계획", "준비"
]


# =========================
# 2) 유틸
# =========================

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; AIcreditResearchBot/1.0; +https://example.org/bot)"
}

def clean_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s or "").strip()
    return s

def year_from_text(text: str) -> Optional[int]:
    """
    본문에서 연도 후보 추출 (예: 2021년, 2021. 등)
    여러 개면 '운영 트리거'와 가까운 연도를 우선 선택.
    """
    if not text:
        return None
    years = [int(y) for y in re.findall(r"(19\d{2}|20\d{2})\s*년?", text)]
    years = [y for y in years if 1990 <= y <= 2035]
    if not years:
        return None
    # 일단 최빈/최소 같은 단순규칙 대신, "운영" 근처를 우선
    # 운영 트리거 등장 위치 기준으로 가까운 연도 선택
    ops_positions = []
    for trig in OPS_TRIGGERS:
        for m in re.finditer(re.escape(trig), text):
            ops_positions.append(m.start())
    if not ops_positions:
        return min(years)
    best = None
    best_dist = 10**9
    for y in set(years):
        for m in re.finditer(str(y), text):
            for p in ops_positions:
                d = abs(m.start() - p)
                if d < best_dist:
                    best_dist = d
                    best = y
    return best or min(years)

def split_sentences_ko(text: str) -> List[str]:
    """
    한국어 문장 분리(완벽하진 않지만 근거문장 뽑기엔 충분)
    """
    text = clean_text(text)
    if not text:
        return []
    # 마침표/물음표/느낌표/다/함/됨 등 기반 분리(간단 버전)
    chunks = re.split(r"(?<=[\.\?\!]|다|함|됨)\s+", text)
    chunks = [c.strip() for c in chunks if len(c.strip()) >= 10]
    return chunks

def has_exclude(text: str) -> bool:
    t = text.lower()
    return any(k.lower() in t for k in EXCLUDE_KEYWORDS)

def has_ai_credit_signal(text: str) -> bool:
    """
    A 변수: 신용/리스크 AI만
    - (AI 토큰 1개 이상) AND (신용/리스크 토큰 1개 이상)
    - 또는 너가 지정한 include 문구(정확/유사)
    """
    t = text
    # 1) exclude 강하게 걸기
    if has_exclude(t):
        return False

    # 2) include 문구 유사매칭(완전일치 안해도 잡기)
    for kw in INCLUDE_KEYWORDS:
        if kw in t:
            return True
        # 유사도(짧은 문구는 과탐 가능하니 80 이상)
        if fuzz.partial_ratio(kw, t) >= 85:
            return True

    # 3) 토큰 조합
    has_ai = any(tok in t for tok in AI_TOKENS)
    has_domain = any(tok in t for tok in CREDIT_RISK_TOKENS)
    return bool(has_ai and has_domain)

def is_operational_claim(text: str) -> bool:
    """
    ‘상용화/운영 중’ 트리거가 있으면 True.
    단, PoC/파일럿/계획 같은 NON_OPS가 강하면 False 우선.
    """
    t = text
    if any(x in t for x in NON_OPS_TRIGGERS):
        # 문장에 PoC/파일럿이 함께 있으면 운영으로 보지 않음(너 규칙)
        # 단, "운영 중"이 명시되면 운영이 이김
        if any(x in t for x in OPS_TRIGGERS):
            return True
        return False
    return any(x in t for x in OPS_TRIGGERS)

def extract_evidence_sentences(text: str, max_evidences: int = 3) -> List[str]:
    """
    근거문장(최대 N개) 뽑기:
    - AI_credit signal 문장
    - 그중 운영 트리거 있는 문장을 최우선
    """
    sents = split_sentences_ko(text)
    candidates = [s for s in sents if has_ai_credit_signal(s)]
    if not candidates:
        return []
    ops = [s for s in candidates if is_operational_claim(s)]
    ordered = ops + [s for s in candidates if s not in ops]
    return ordered[:max_evidences]


# =========================
# 3) 본문 추출
# =========================

def fetch_url(url: str, timeout: int = 20) -> Optional[str]:
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        if r.status_code != 200:
            return None
        r.encoding = r.apparent_encoding
        return r.text
    except Exception:
        return None

def extract_main_text(url: str, html: Optional[str] = None) -> str:
    """
    trafilatura로 메인 텍스트 추출
    """
    if html is None:
        html = fetch_url(url)
    if not html:
        return ""
    downloaded = trafilatura.extract(html, url=url, include_comments=False, include_tables=False)
    return clean_text(downloaded or "")


# =========================
# 4) 공식 사이트/보도자료 URL 수집
#   - sitemap.xml 우선
#   - 안 되면 seed 목록 페이지에서 링크 수집
# =========================

def parse_sitemap_urls(sitemap_url: str, limit: int = 5000) -> List[str]:
    xml = fetch_url(sitemap_url)
    if not xml:
        return []
    soup = BeautifulSoup(xml, "xml")
    locs = [loc.get_text(strip=True) for loc in soup.find_all("loc")]
    return locs[:limit]

def collect_links_from_listing(listing_url: str, domain_filter: Optional[str] = None, limit: int = 300) -> List[str]:
    html = fetch_url(listing_url)
    if not html:
        return []
    soup = BeautifulSoup(html, "lxml")
    links = []
    for a in soup.select("a[href]"):
        href = a.get("href")
        if not href:
            continue
        full = urllib.parse.urljoin(listing_url, href)
        if domain_filter and urllib.parse.urlparse(full).netloc and domain_filter not in full:
            continue
        links.append(full)
    # 중복 제거
    uniq = []
    seen = set()
    for u in links:
        if u not in seen:
            seen.add(u)
            uniq.append(u)
    return uniq[:limit]


# =========================
# 5) 뉴스 수집 (권장: Google News RSS)
# =========================

def google_news_rss(query: str, hl: str = "ko", gl: str = "KR", ceid: str = "KR:ko") -> str:
    q = urllib.parse.quote(query)
    return f"https://news.google.com/rss/search?q={q}&hl={hl}&gl={gl}&ceid={ceid}"

def collect_news_urls_google_rss(query: str, max_items: int = 50) -> List[Tuple[str, Optional[str]]]:
    """
    return: [(url, published_str)]
    """
    rss_url = google_news_rss(query)
    feed = feedparser.parse(rss_url)
    out = []
    for e in feed.entries[:max_items]:
        link = e.get("link")
        published = e.get("published") or e.get("updated")
        if link:
            out.append((link, published))
    return out

# (옵션) GDELT 2.1: 훨씬 많이 잡히는데, 한국어 커버리지 편차 있음
def collect_news_urls_gdelt(query: str, max_records: int = 100) -> List[str]:
    base = "https://api.gdeltproject.org/api/v2/doc/doc"
    params = {
        "query": query,
        "mode": "ArtList",
        "format": "json",
        "maxrecords": str(max_records),
        "sort": "HybridRel"
    }
    url = base + "?" + urllib.parse.urlencode(params)
    js = None
    try:
        r = requests.get(url, headers=HEADERS, timeout=20)
        if r.status_code != 200:
            return []
        js = r.json()
    except Exception:
        return []
    arts = js.get("articles", []) if isinstance(js, dict) else []
    return [a.get("url") for a in arts if a.get("url")]


# =========================
# 6) 결과 스키마
# =========================

@dataclass
class EvidenceRow:
    bank: str
    source_type: str     # "official" / "press" / "news"
    url: str
    title: str
    published: str
    year_operational_first: Optional[int]
    ai_credit_mention: int     # 1/0
    operational_claim: int     # 1/0
    evidence_1: str
    evidence_2: str
    evidence_3: str
    notes: str


# =========================
# 7) 파이프라인
# =========================

def analyze_url(bank: str, url: str, source_type: str, published: Optional[str] = None) -> EvidenceRow:
    html = fetch_url(url)
    title = ""
    if html:
        soup = BeautifulSoup(html, "lxml")
        if soup.title and soup.title.get_text():
            title = clean_text(soup.title.get_text())
    text = extract_main_text(url, html=html)

    evidences = extract_evidence_sentences(text, max_evidences=3)
    ai_mention = 1 if len(evidences) > 0 else 0
    ops_claim = 1 if any(is_operational_claim(e) for e in evidences) else 0

    # 운영/상용화 최초 연도: 운영 트리거 문장 중심으로 추정
    year = None
    if ops_claim:
        year = year_from_text(" ".join([e for e in evidences if is_operational_claim(e)])) or year_from_text(text)
    else:
        year = None

    # published 파싱 (RSS 등에서 받은 값이 있으면 사용)
    pub_str = clean_text(published or "")
    if pub_str:
        dt = dateparser.parse(pub_str, languages=["ko", "en"])
        if dt:
            pub_str = dt.strftime("%Y-%m-%d")

    return EvidenceRow(
        bank=bank,
        source_type=source_type,
        url=url,
        title=title,
        published=pub_str,
        year_operational_first=year,
        ai_credit_mention=ai_mention,
        operational_claim=ops_claim,
        evidence_1=evidences[0] if len(evidences) > 0 else "",
        evidence_2=evidences[1] if len(evidences) > 1 else "",
        evidence_3=evidences[2] if len(evidences) > 2 else "",
        notes=""
    )

def run_official_press_crawl(bank: str, seeds: Dict[str, List[str]], sleep_sec: float = 0.8, max_urls: int = 300) -> List[EvidenceRow]:
    """
    seeds 예시:
    {
      "sitemaps": ["https://.../sitemap.xml", ...],
      "listings": ["https://.../press", "https://.../news", ...],
      "domain": "example.com"
    }
    """
    domain = seeds.get("domain", "")
    urls = []

    # 1) sitemap 기반 수집
    for sm in seeds.get("sitemaps", []):
        urls.extend(parse_sitemap_urls(sm, limit=max_urls))

    # 2) listing 기반 수집(공지/보도자료)
    for lst in seeds.get("listings", []):
        urls.extend(collect_links_from_listing(lst, domain_filter=domain, limit=max_urls))

    # 정리 + 도메인 필터 + 중복제거
    filtered = []
    seen = set()
    for u in urls:
        if domain and domain not in u:
            continue
        if u not in seen:
            seen.add(u)
            filtered.append(u)

    filtered = filtered[:max_urls]

    rows = []
    for u in tqdm(filtered, desc=f"[{bank}] official/press crawl"):
        row = analyze_url(bank, u, source_type="official")
        if row.ai_credit_mention == 1:
            # 신호 있는 것만 저장(노이즈 줄이기)
            rows.append(row)
        time.sleep(sleep_sec)
    return rows

def run_news_crawl(bank: str, sleep_sec: float = 0.8, per_query: int = 30) -> List[EvidenceRow]:
    """
    뉴스는 키워드 쿼리로 잡고, 본문에서 AI_credit만 남김.
    """
    queries = [
        f"{bank} 인공지능 신용평가 상용화",
        f"{bank} AI 대출심사 운영",
        f"{bank} 머신러닝 리스크관리 운영",
        f"{bank} FDS 고도화 AI 운영",
        f"{bank} AML AI 시스템 운영",
    ]
    seen = set()
    rows = []

    for q in queries:
        items = collect_news_urls_google_rss(q, max_items=per_query)
        for (url, published) in items:
            if url in seen:
                continue
            seen.add(url)
            row = analyze_url(bank, url, source_type="news", published=published)
            if row.ai_credit_mention == 1:
                rows.append(row)
            time.sleep(sleep_sec)

    return rows


# =========================
# 8) 은행별 seed 템플릿
# =========================

BANK_SEEDS = {
    "국민은행": {
        "domain": "kbstar.com",
        "sitemaps": [],
        "listings": ["https://omoney.kbstar.com/quics?page=C017648"]
    },
    "신한은행": {
        "domain": "shinhan.com",
        "sitemaps": [],
        "listings": ["https://www.shinhan.com/kr/press/pressList.html"]
    },
    "우리은행": {
        "domain": "wooribank.com",
        "sitemaps": [],
        "listings": ["https://spot.wooribank.com/pot/Dream?withyou=BPPBC0036"]
    },
    "기업은행": {
        "domain": "ibk.co.kr",
        "sitemaps": [],
        "listings": ["https://www.ibk.co.kr/IBK/notification/pressReleaseList.do"]
    },
    "하나은행": {
        "domain": "kebhana.com",
        "sitemaps": [],
        "listings": ["https://www.kebhana.com/cont/mall/mall15/index.jsp?MIDX=20959"]
    },
    "농협은행": {
        "domain": "nhbank.com",
        "sitemaps": [],
        "listings": ["https://www.nhbank.com/nhbank/introduce/press/pressList.do"]
    },
    "케이뱅크": {
        "domain": "kbanknow.com",
        "sitemaps": [],
        "listings": ["https://kbanknow.com/notice/list.do?menuId=090000"]
    },
    "토스뱅크": {
        "domain": "tossbank.com",
        "sitemaps": [],
        "listings": ["https://tossbank.com/notice"]
    },
    "카카오뱅크": {
        "domain": "kakaobank.com",
        "sitemaps": [],
        "listings": ["https://www.kakaobank.com/Corp/News/PressRelease"]
    }
}


def main(
    banks: Optional[List[str]] = None,
    out_csv: str = "ai_credit_web_evidence.csv"
):
    if banks is None:
        banks = list(BANK_SEEDS.keys())

    all_rows: List[EvidenceRow] = []

    for bank in banks:
        seeds = BANK_SEEDS.get(bank, {})
        # 1) 공식/보도자료
        if seeds:
            all_rows.extend(run_official_press_crawl(bank, seeds, sleep_sec=0.8, max_urls=250))

        # 2) 뉴스
        all_rows.extend(run_news_crawl(bank, sleep_sec=0.8, per_query=25))

    # dataframe
    df = pd.DataFrame([r.__dict__ for r in all_rows])

    # 동일 URL 중복 제거
    if not df.empty:
        df = df.drop_duplicates(subset=["url"]).reset_index(drop=True)

        # 운영 주장(operational_claim=1) 우선 정렬
        df = df.sort_values(by=["bank", "operational_claim", "year_operational_first"], ascending=[True, False, True])

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"[DONE] saved: {out_csv}  (rows={len(df)})")


if __name__ == "__main__":
    main()


[국민은행] official/press crawl: 100%|██████████| 153/153 [03:36<00:00,  1.42s/it]
[신한은행] official/press crawl: 0it [00:00, ?it/s]
[우리은행] official/press crawl: 100%|██████████| 70/70 [01:29<00:00,  1.27s/it]
[기업은행] official/press crawl: 0it [00:00, ?it/s]
[농협은행] official/press crawl: 100%|██████████| 7/7 [00:08<00:00,  1.20s/it]
[케이뱅크] official/press crawl: 0it [00:00, ?it/s]
[카카오뱅크] official/press crawl: 100%|██████████| 13/13 [00:15<00:00,  1.19s/it]


[DONE] saved: ai_credit_web_evidence.csv  (rows=7)
